# Task A

In this task, we had to create a "transform" function which transforms unperformed beats time into a performed version.

In other words, given the `midi_score_annotations.txt` and `NAME_annotations.txt`, we need to create a function that transfers first column of the first file into the first column of the second file.

For the velocity, we use `*.mid` files and extract velocity from events. "Beat velocity" is defined as an average of the velocities of the events close to this beat. Event is close to the beat iff the difference between its onset in score and beat time is smaller than `0.5`. Then we again create a "transform" function.

In [ ]:
%load_ext autoreload
%autoreload 2

from src.data import *
from src.plots import *
from src.estimators import *
from src.onset_distribution import *

import seaborn
seaborn.set_style("whitegrid")

In [ ]:
df, json_data = get_dataset_metadata("Bach")

In [ ]:
df.head(2)

In [ ]:
# for the first time might take up to 5-10 min because of velocity
beats_list_dict = get_midi_performance_pairs(df, json_data, "4/4")

In [ ]:
train_beats_list_dict, test_beats_list_dict = train_test_split(beats_list_dict, test_size=0.2)

In [ ]:
print("Train / Test:", len(train_beats_list_dict["midi_beats_list"]), "/", len(test_beats_list_dict["midi_beats_list"]))

In [ ]:
midi_beats_list = test_beats_list_dict["midi_beats_list"]
velocity_beats_list = test_beats_list_dict["velocity_beats_list"]
performance_beats_list = test_beats_list_dict["performance_beats_list"]
perf_velocity_beats_list = test_beats_list_dict["perf_velocity_beats_list"]

performance_beats_estimated_list_dict = {}
perf_velocity_beats_estimated_list_dict = {}

In [ ]:
# random estimate
performance_beats_estimated_list, velocity_beats_estimated_list = get_estimator_predictions(train_beats_list_dict,
                                                                                            test_beats_list_dict,
                                                                                            estimator_type="random")

performance_beats_estimated_list_dict["random"] = performance_beats_estimated_list
perf_velocity_beats_estimated_list_dict["random"] = velocity_beats_estimated_list

In [ ]:
# linear estimate
performance_beats_estimated_list, velocity_beats_estimated_list = get_estimator_predictions(train_beats_list_dict,
                                                                                            test_beats_list_dict,
                                                                                            estimator_type="linear")

performance_beats_estimated_list_dict["linear"] = performance_beats_estimated_list
perf_velocity_beats_estimated_list_dict["linear"] = velocity_beats_estimated_list

In [ ]:
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

warnings.filterwarnings("ignore", category=RuntimeWarning) 


fig, axes = plt.subplots(1, 2, figsize=(9, 4))
axes[0] = plot_average_transfer_function(axes[0], midi_beats_list,
                                        midi_beats_list, # unperformed_beats_list
                                        performance_beats_list,
                                        performance_beats_estimated_list_dict,
                                        performance_type="time"
)

axes[1] = plot_average_transfer_function(axes[1], midi_beats_list,
                                        velocity_beats_list, # unperformed_beats_list
                                        perf_velocity_beats_list,
                                        perf_velocity_beats_estimated_list_dict,
                                        performance_type="velocity"
)
plt.savefig("plots/task_A.pdf", dpi=600)

# Task B

Use `src.data.get_events_table_from_score`

In [ ]:
from pathlib import Path
# We analyse Beethoven Sonatas
corpus = DATASET_PATH / "Beethoven" / "Piano_Sonatas"

In [ ]:

unique_signatures = list_all_time_signatures(corpus)
sigs = [
    ((4,4), 4),
    ((3,4), 4),
    ((6,8), 2)
]

results = [(get_average_distribution_given_time_signature(corpus, sig[0], sig[1]), sig[0]) for sig in sigs]

In [ ]:
plot_beat_frequencies(results, figsize=(16, 4))
plt.savefig("plots/beethov_combined.pdf", dpi=600)

In [ ]:
styles = ["Baroque", "Classical", "Romantism", "Impressionism", "20th Century Russian"]
styles_composers = [baroque_composers, classical_composers, romantic_composers, impressionist_composers, late_russian_composers]
composers = []
for style_composers in styles_composers:
    composers.extend(style_composers)
expr_by_composer = composer_expressiveness_analysis(composers)
expr_by_style = style_expressiveness_analysis(styles_composers)


In [ ]:
plot_composer_and_style(composers, styles, expr_by_composer, expr_by_style)

In [ ]:
for time_signature in [(4,4), (3,4), (6,8)]:
    beat = time_signature[0]
    bar = time_signature[1]
    pieces = collect_pieces_with_time_signature(corpus, [beat, bar])
    beat_deviations = combine_beat_deviations(pieces, beat)
    plot_violins(beat_deviations, f'Beethoven Piano Sonatas {time_signature}', (-100, 100))
    

In [ ]:
# Check where high deviation values (>2) are coming from
# for piece in pieces:
#     interval = annotation_to_inter_onset_intervals(piece)
#     deviation = (interval - interval.median())/interval.median()
#     if sum(deviation[deviation > 2]):
#         print(piece.split('/')[-4:])
#         print(deviation[deviation > 2])
#         input('press enter to continue')

        

In [ ]:
pieces = collect_pieces_with_time_signature(corpus, [4, 4])
beat_deviations = combine_beat_deviations(pieces, beat)

In [ ]:
beat_deviations[beat_deviations['Beat']==0]['Deviation'].median()
beat_deviations[beat_deviations['Beat']==1]['Deviation'].median()

In [ ]:
import seaborn
seaborn.set_style("whitegrid")

fig, axes = plt.subplots(1, 3, figsize=(12, 4)) # think about figsize
for time_signature, axes in zip([(4,4), (3,4), (6,8)], axes):
    beat = time_signature[0]
    bar = time_signature[1]
    pieces = collect_pieces_with_time_signature(corpus, [beat, bar])
    beat_deviations = combine_beat_deviations(pieces, beat)
    plot_violins(beat_deviations, f'Time signature{time_signature}', (-100, 100), axes)

#fig.suptitle('IOI Deviations by Time Signature for Beetoven Piano Sonatas',y=1.05)
fig.tight_layout()
plt.savefig("plots/violin_plots.pdf", dpi=600)